In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator

from data_generator import RandomCropGenerator
from model_builder import ModelBuilder

In [ ]:
COLUMNS = ['ImageFile', 'POS_X', 'POS_Y', 'POS_Z', 'Q_W', 'Q_X', 'Q_Y', 'Q_Z']
IMG_COLUMN = 'ImageFile'
POSITION_COLUMNS = ['POS_X', 'POS_Y', 'POS_Z']
ORIENTATION_COLUMNS = ['Q_W', 'Q_X', 'Q_Y', 'Q_Z']

DIRECTORY = 'dataset'
DATA_PATH = 'info.csv'
COLUMN_SEP = ','

IMG_SIZE = (256, 256)
IMG_CROP_SIZE = (224, 224)
POSE_REGRESSOR_SIZE = 2048
BATCH_SIZE = 50
SHUFFLE = True
RANDOM_SEED = 4444

### Read and split data

In [7]:
np.random.seed(RANDOM_SEED)
data = pd.read_csv(DATA_PATH, sep=COLUMN_SEP, usecols=COLUMNS)
train_data, test_data = train_test_split(data, test_size=0.1)
print(f'train size: {train_data.shape[0]}, test_size: {test_data.shape[0]}')
train_data.head()

train size: 900, test_size: 100


,POS_X,POS_Y,POS_Z,Q_W,Q_X,Q_Y,Q_Z,ImageFile
435,1.404367,0.621548,-0.038781,0.926828,0.0,-0.0,-0.375486,img_0_0_1542103551932911900.png
26,-2.294151,-2.619398,-0.038781,0.992391,0.0,-0.0,-0.123129,img_0_0_1542099351874224500.png
456,1.038382,0.173967,-0.038781,0.800721,-0.0,0.0,0.599037,img_0_0_1542103707070959200.png
125,0.748456,-0.351675,-0.038781,0.225952,-0.0,0.0,0.974138,img_0_0_1542100318797415700.png
690,1.482945,-1.242870,-0.038781,1.000000,0.0,-0.0,-0.000002,img_0_0_1542105894912803500.png


In [8]:
train_data.describe()

,POS_X,POS_Y,POS_Z,Q_W,Q_X,Q_Y,Q_Z
count,900.000000,900.000000,9.000000e+02,900.000000,900.0,900.0,900.000000
mean,0.040090,-1.369665,-3.878100e-02,0.709123,0.0,0.0,-0.006197
std,1.483758,1.256425,1.596833e-16,0.317153,0.0,0.0,0.630137
min,-3.077480,-3.778143,-3.878100e-02,0.000018,0.0,-0.0,-1.000000
25%,-1.285080,-2.529762,-3.878100e-02,0.474060,0.0,0.0,-0.535099
50%,0.154681,-1.205611,-3.878100e-02,0.839133,0.0,-0.0,0.002253
75%,1.440197,-0.358070,-3.878100e-02,0.985918,0.0,0.0,0.556028
max,2.812647,1.622524,-3.878100e-02,1.000000,0.0,-0.0,1.000000


### Instantiate data generators

In [9]:
imagegen = ImageDataGenerator()
traingen = RandomCropGenerator(
    directory=DIRECTORY, dataframe=train_data, image_data_generator=imagegen,
    x_col=IMG_COLUMN, y_col=POSITION_COLUMNS+ORIENTATION_COLUMNS,
    target_size=IMG_SIZE, random_crop_shape=IMG_CROP_SIZE,
    class_mode='raw', batch_size=BATCH_SIZE, shuffle=SHUFFLE)

testgen = RandomCropGenerator(
    directory=DIRECTORY, dataframe=test_data, image_data_generator=imagegen,
    x_col=IMG_COLUMN, y_col=POSITION_COLUMNS+ORIENTATION_COLUMNS,
    target_size=IMG_SIZE, random_crop_shape=IMG_CROP_SIZE,
    class_mode='raw', batch_size=BATCH_SIZE, shuffle=SHUFFLE)

Found 900 validated image filenames.
Found 100 validated image filenames.


### Estimate pose coordinates ranges

In [10]:
position_max = train_data[POSITION_COLUMNS+ORIENTATION_COLUMNS].values.max(axis=0)
position_min = train_data[POSITION_COLUMNS+ORIENTATION_COLUMNS].values.min(axis=0)
position_ranges = position_max - position_min
position_ranges

array([5.890127, 5.400667, 0.      , 0.999982, 0.      , 0.      ,
       2.      ])

### Build a model

In [ ]:
model_builder = ModelBuilder()

In [ ]:
model = model_builder.build_model(image_size=IMG_CROP_SIZE, pose_regressor_size=POSE_REGRESSOR_SIZE,
                                  position_ranges=position_ranges)

In [16]:
model_builder.fit_model(model, traingen, testgen,
                        epochs=40, lr=0.001, lr_decay_rate=0.1, lr_decay_step=10, min_lr=0.00005,
                        beta=20)

Epoch 1/40
18/18 [==============================] - 37s 2s/step - loss: 16.5511 - xyz_loss: 2.1392 - wpqr_loss: 0.6871 - wpqr_orientation_angle: 72.5042 - val_loss: 21.6138 - val_xyz_loss: 5.9530 - val_wpqr_loss: 0.7508 - val_wpqr_orientation_angle: 81.6019
Epoch 2/40
18/18 [==============================] - 21s 1s/step - loss: 7.9094 - xyz_loss: 1.7026 - wpqr_loss: 0.2792 - wpqr_orientation_angle: 30.0493 - val_loss: 14.0377 - val_xyz_loss: 2.6308 - val_wpqr_loss: 0.5404 - val_wpqr_orientation_angle: 59.8488
Epoch 3/40
18/18 [==============================] - 21s 1s/step - loss: 5.8842 - xyz_loss: 1.4021 - wpqr_loss: 0.1951 - wpqr_orientation_angle: 21.0208 - val_loss: 9.6560 - val_xyz_loss: 2.2514 - val_wpqr_loss: 0.3424 - val_wpqr_orientation_angle: 38.3417
Epoch 4/40
18/18 [==============================] - 21s 1s/step - loss: 5.2722 - xyz_loss: 1.2291 - wpqr_loss: 0.1752 - wpqr_orientation_angle: 18.3552 - val_loss: 11.2452 - val_xyz_loss: 1.4321 - val_wpqr_loss: 0.4647 - val_wpqr